# Shortest Path Example

9.3-1. You need to take a trip by car to another town that you have never visited before. Therefore, you are studying a map to determine the shortest route to your destination. Depending on which route you choose, there are five other towns (call them A, B, C, D, E) that you might pass through on the way. The map
shows the mileage along each road that directly connects two towns without any intervening towns. These numbers are sum- marized in the following table, where a dash indicates that there is no road directly connecting these two towns without going through any other towns.

This problem is from Chapter 3, Section 1 (3.1 Prototype Example) of the Introduction to Operations Research,
7th edition by Hillier and Lieberman.

<div>
<img src="img_9.3-1_table.png" width="400"/>
</div>

### Graph Notation

<!-- ![title](img_9.3-1_network.png) -->
<div>
<img src="img_9.3-1_network.png" width="700"/>
</div>

Let V represent the set of vertices in the graph:
$$ V =\{O,A,B,C,D,E,T\} $$

Let E represent the set of edges in the graph:
$$ 
E = \{OA, OB, OC, AB, AD, BD, BE, CB, CE, DE, DT, ET\}
$$

Let $u$ represent the first vertice and $v$ represent the second vertice in edge $UV$ in the path $UV-VW$ (U->V->W)

Let $v$ represent the first vertice and $w$ represent the second vertice in edge $VW$ in the path $UV-VW$ (U->V->W)

Let $x_{uv}$ $\epsilon$ $(0,1)$
<br>$x_{uv}$ = 1 if edge ${uv}$ is used, 0 otherwise.

Let $d_{uv}$ represent the distance between edge $u$ and $v$.

### Mathematical Formulation

$Minimize: Z $



$$Z = \sum_{u,v \: \epsilon \:E} x_{uv}$$

$ 
s.t: 
$

Constraint 1: Ensures path starts at origin and that each subsequent edge in the path is a continuation from the previous edge. In other words, if the trip enters into a given city (excluding final destination), it must leave the same city on the next leg of the path. 
<br>
<br>
$$
\sum_{u,v \: \epsilon \:E} x_{uv} = 
\sum_{v,w \: \epsilon \:E} x_{uv} \qquad 
\forall  \enspace v \: \epsilon \:V
$$ 



Constraint 2: Ensures the path reaches the final destination. 
<br>
<br>
$$
\sum_{u,t \: \epsilon \:E} x_{ut} = 1 
$$

# DOCplex.mp Python Model

### Install packages (only if needed)

Import using conda in the current Jupyter kernel

In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} matplotlib
# !conda install --yes --prefix {sys.prefix} docplex

Or alternitively, import a using pip

In [2]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install docplex

### Import packages

In [3]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import docplex.mp.solution as Solution

### Intializing the data

In [4]:
# Initialize the problem data
df = pd.read_csv('data_arc.csv')
df

,origin,destination,OD_pair,distance
0,O,A,OA,40
1,O,C,OC,60
2,O,B,OB,50
3,A,B,AB,10
4,A,D,AD,70
5,B,D,BD,20
6,B,E,BE,55
7,C,B,CB,40
8,C,E,CE,50
9,D,E,DE,10


In [5]:
edges = list((t.origin, t.destination) for t in df.titertuples())
edges

[('O', 'A'),
 ('O', 'C'),
 ('O', 'B'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'D'),
 ('B', 'E'),
 ('C', 'B'),
 ('C', 'E'),
 ('D', 'E'),
 ('D', 'T'),
 ('E', 'T')]

In [6]:
last_arc_df = df[df['destination'] == 'T']
last_arc = list((t.origin, t.destination) for t in last_arc_df.itertuples())
last_arc

[('D', 'T'), ('E', 'T')]

In [7]:
final_destination = 'T'
cities = set(df['destination'])
cities.remove(final_destination)
cities

{'A', 'B', 'C', 'D', 'E'}

In [8]:

# cities = ['A', 'B','C', 'D', 'E']


In [9]:
distance = dict([((t.origin, t.destination),t.distance ) for t in df.itertuples()])
distance

{('O', 'A'): 40,
 ('O', 'C'): 60,
 ('O', 'B'): 50,
 ('A', 'B'): 10,
 ('A', 'D'): 70,
 ('B', 'D'): 20,
 ('B', 'E'): 55,
 ('C', 'B'): 40,
 ('C', 'E'): 50,
 ('D', 'E'): 10,
 ('D', 'T'): 60,
 ('E', 'T'): 80}

### Create the model

In [10]:
m = Model('Shortest_Path')

recall:
Let $x_{uv}$ $\epsilon$ $(0,1)$
<br>$x_{uv}$ = 1 if edge ${uv}$ is used, 0 otherwise.

In [11]:
# Create decision variables
x = m.binary_var_dict(edges, name = 'x')


#### <font color=green>  Objective Function

recall: Minimize Z $$Z = \sum_{u,v } x_{uv}$$

In [12]:
m.minimize(m.sum(distance[uv]*x[uv] for uv in edges))
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Shortest_Path

Minimize
 obj: 40 x_O_A + 60 x_O_C + 50 x_O_B + 10 x_A_B + 70 x_A_D + 20 x_B_D + 55 x_B_E
      + 40 x_C_B + 50 x_C_E + 10 x_D_E + 60 x_D_T + 80 x_E_T
Subject To

Bounds
 0 <= x_O_A <= 1
 0 <= x_O_C <= 1
 0 <= x_O_B <= 1
 0 <= x_A_B <= 1
 0 <= x_A_D <= 1
 0 <= x_B_D <= 1
 0 <= x_B_E <= 1
 0 <= x_C_B <= 1
 0 <= x_C_E <= 1
 0 <= x_D_E <= 1
 0 <= x_D_T <= 1
 0 <= x_E_T <= 1

Binaries
 x_O_A x_O_C x_O_B x_A_B x_A_D x_B_D x_B_E x_C_B x_C_E x_D_E x_D_T x_E_T
End



#### <font color=green>  Constraints

Recall: Constraint 1:
$$
\sum_{u,v \: \epsilon \:E} x_{uv} = 
\sum_{v,w \: \epsilon \:E} x_{uv} \qquad 
\forall  \enspace v \: \epsilon \: Cities
$$ 



Recall: Constraint 2:
$$
\sum_{u,t \: \epsilon \:E} x_{ut} = 1 
$$

In [13]:
# constraint 1
for c in cities:
    m.add_constraint((m.sum(x[(u,v)] for u,v in edges if v==c))== 
                     (m.sum(x[(v,w)] for v,w in edges if v==c)), ctname='city_'+ c)

# constraint 2
m.add_constraint(m.sum(x[(i,j)] for i,j in last_arc if j=='T')== 1, ctname='last_arc')

docplex.mp.LinearConstraint[last_arc](x_D_T+x_E_T,EQ,1)

In [14]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Shortest_Path

Minimize
 obj: 40 x_O_A + 60 x_O_C + 50 x_O_B + 10 x_A_B + 70 x_A_D + 20 x_B_D + 55 x_B_E
      + 40 x_C_B + 50 x_C_E + 10 x_D_E + 60 x_D_T + 80 x_E_T
Subject To
 city_A: x_O_A - x_A_B - x_A_D = 0
 city_C: x_O_C - x_C_B - x_C_E = 0
 city_D: x_A_D + x_B_D - x_D_E - x_D_T = 0
 city_E: x_B_E + x_C_E + x_D_E - x_E_T = 0
 city_B: x_O_B + x_A_B + x_C_B - x_B_D - x_B_E = 0
 last_arc: x_D_T + x_E_T = 1

Bounds
 0 <= x_O_A <= 1
 0 <= x_O_C <= 1
 0 <= x_O_B <= 1
 0 <= x_A_B <= 1
 0 <= x_A_D <= 1
 0 <= x_B_D <= 1
 0 <= x_B_E <= 1
 0 <= x_C_B <= 1
 0 <= x_C_E <= 1
 0 <= x_D_E <= 1
 0 <= x_D_T <= 1
 0 <= x_E_T <= 1

Binaries
 x_O_A x_O_C x_O_B x_A_B x_A_D x_B_D x_B_E x_C_B x_C_E x_D_E x_D_T x_E_T
End



### Solve

In [15]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

soln = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Tried aggregator 2 times.
MIP Presolve added 1 rows and 1 columns.
Aggregator did 1 substitutions.
Reduced MIP has 6 rows, 12 columns, and 22 nonzeros.
Reduced MIP has 11 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Found incumbent of value 160.000000 after 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 6 rows, 12 columns, and 22 nonzeros.
Reduced MIP has 12 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing 

In [21]:
print(m.get_solve_status())


JobSolveStatus.OPTIMAL_SOLUTION


In [17]:
soln.display()

solution for: Shortest_Path
objective: 130
x_O_A = 1
x_A_B = 1
x_B_D = 1
x_D_T = 1


In [18]:
lst = []
for i in x:
    if soln.get_var_value(x[i]) >0:
        lst.append(i[1])
lst



['A', 'B', 'D', 'T']

In [48]:
lst = []
for u,v in edges:
    if soln.get_var_value(x[u,v])> 0:
        arc = u+'->'+v
        print(arc)
        solution = (u, v, arc,distance[u,v])
        lst.append(solution)
lst


O->A
A->B
B->D
D->T


[('O', 'A', 'O->A', 40),
 ('A', 'B', 'A->B', 10),
 ('B', 'D', 'B->D', 20),
 ('D', 'T', 'D->T', 60)]

In [51]:
df = pd.DataFrame.from_records(lst, columns=['starting_city', 'next_city','arc_name','distance'])
df

,starting_city,next_city,arc_name,distance
0,O,A,O->A,40
1,A,B,A->B,10
2,B,D,B->D,20
3,D,T,D->T,60
